In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# GCP 上的 E2E ML：MLOps 阶段 2：实验：使用 Vertex AI Training 开始使用 Scikit-Learn

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage2/get_started_vertex_training_sklearn.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage2/get_started_vertex_training_sklearn.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage2/get_started_vertex_training_sklearn.ipynb">
     <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
在 Vertex AI Workbench 中打开
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何在谷歌云上使用Vertex AI进行端到端MLOps生产。本教程涵盖了阶段2：实验：从头开始使用Vertex AI Training进行scikit-Learn。

### 目标

在本教程中，您将学习如何使用 `Vertex AI Training` 来训练一个 Scikit-Learn 自定义模型。

本教程使用以下 Google Cloud ML 服务：

- `Vertex AI Training`
- `Vertex AI Model` 资源

执行的步骤包括：

- 使用 Python 包进行训练。
- 在超参数调整时报告准确率。
- 使用 GCSFuse 将模型工件保存到 Cloud Storage。
- 创建一个 `Vertex AI Model` 资源。

数据集

本教程使用的数据集是来自[ICS机器学习数据集](https://archive.ics.uci.edu/ml/datasets.php)的[新闻聚合](https://archive.ics.uci.edu/ml/datasets/News+Aggregator)。训练好的模型可以预测新闻文章的新闻类别。

费用

本教程使用 Google Cloud 的计费组件：

* Vertex AI
* Cloud Storage

了解 [Vertex AI 的定价](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage 的定价](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您的预期使用情况生成费用估算。

### 设置本地开发环境

**如果您正在使用Colab或Vertex AI Workbench笔记本**，您的环境已经满足运行此笔记本的所有要求。您可以跳过这一步。

**否则**，确保您的环境满足这个笔记本的要求。您需要以下内容：

* Google Cloud SDK
* Git
* Python 3
* virtualenv
* 在使用Python 3的虚拟环境中运行的Jupyter笔记本

Google Cloud指南 [设置Python开发环境](https://cloud.google.com/python/setup) 和 [Jupyter安装指南](https://jupyter.org/install) 提供了满足这些要求的详细说明。以下步骤提供了一套简化的说明：

1. [安装并初始化Cloud SDK。](https://cloud.google.com/sdk/docs/)

2. [安装Python 3。](https://cloud.google.com/python/setup#installing_python)

3. [安装virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv) 并创建一个使用Python 3的虚拟环境。激活虚拟环境。

4. 要安装Jupyter，在终端Shell上的命令行中运行 `pip3 install jupyter`。

5. 要启动Jupyter，在终端Shell上的命令行中运行 `jupyter notebook`。

6. 在Jupyter Notebook仪表板中打开该笔记本。

## 安装

安装以下软件包以执行此笔记本。

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME") and not os.getenv("VIRTUAL_ENV")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install {USER_FLAG} --upgrade google-cloud-aiplatform -q

### 重新启动内核

安装了额外的软件包后，您需要重新启动笔记本内核，以便它可以找到这些软件包。

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 在开始之前

### 设置您的Google Cloud项目

**无论您使用的是哪种笔记本环境，都需要执行以下步骤。**

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建帐户时，您将获得300美元的免费信用额度，用于支付计算/存储成本。

2. [确保您的项目已启用计费功能](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [启用Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。

4. 如果您在本地运行此笔记本，您需要安装[Cloud SDK](https://cloud.google.com/sdk)。

5. 在下面的单元格中输入您的项目ID。然后运行该单元格，以确保Cloud SDK为本笔记本中的所有命令使用正确的项目。

**注意**：Jupyter会将以`!`开头的行视为shell命令，并将以`$`开头的Python变量插入这些命令中。

设置您的项目ID

**如果您不知道您的项目ID**，您可以使用`gcloud`来获取您的项目ID。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 区域

您还可以更改`REGION`变量，该变量用于整个笔记本的操作。以下是 Vertex AI 支持的区域。我们建议您选择最靠近您的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太：`asia-east1`

您可能不使用多区域存储桶来训练 Vertex AI。并非所有区域都支持所有 Vertex AI 服务。

了解更多关于 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

时间戳

如果您正在进行实时教程会话，可能会使用共享测试帐户或项目。为了避免用户在创建的资源上发生名称冲突，您需要为每个实例会话创建一个时间戳，并将时间戳附加到您在本教程中创建的资源名称中。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的 Google Cloud 帐户

**如果您正在使用 Vertex AI Workbench 笔记本**，则您的环境已经经过身份验证。请跳过此步骤。

**如果您正在使用 Colab**，运行下面的单元格，并按照提示进行验证您的帐户。

**否则**，请按照以下步骤操作：

在 Cloud 控制台中，转到 [创建服务帐号密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey) 页面。

1. **点击创建服务帐号**。

2. 在“服务帐号名称”字段中输入名称，然后点击**创建**。

3. 在“授予此服务帐号对项目的访问权限”部分，单击角色下拉列表。在筛选框中键入“Vertex AI”，然后选择**Vertex AI 管理员**。在筛选框中键入“存储对象管理员”，然后选择**存储对象管理员**。

4. 点击创建。包含您的密钥的 JSON 文件将下载到本地环境中。

5. 在下面的单元格中将您的服务帐号密钥的路径输入为 GOOGLE_APPLICATION_CREDENTIALS 变量，然后运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = False
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        IS_COLAB = True
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您使用哪种笔记本环境，下面的步骤都是必需的。**

当您初始化用于 Python 的 Vertex SDK 时，您需要指定一个云存储暂存桶。这个暂存桶是您的数据集和模型资源在不同工作会话中保留的地方。

在下面设定您的云存储桶的名称。存储桶名称必须在所有谷歌云项目中是全局唯一的，包括您组织外的项目。

In [ ]:
BUCKET_URI = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_URI == "" or BUCKET_URI is None or BUCKET_URI == "gs://[your-bucket-name]":
    BUCKET_URI = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

只有当您的存储桶尚不存在时，才运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

最后，通过检查云存储桶的内容来验证对其的访问。

In [ ]:
! gsutil ls -al $BUCKET_URI

### 设置变量

接下来，设置一些在整个教程中将要使用的变量。
### 导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aip

### 初始化Vertex AI SDK for Python

为您的项目和相应的存储桶初始化Vertex AI SDK。

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

设置硬件加速器

您可以为训练和预测设置硬件加速器。

设置变量 `TRAIN_GPU/TRAIN_NGPU` 和 `DEPLOY_GPU/DEPLOY_NGPU`，以使用支持 GPU 的容器映像和分配给虚拟机实例（VM）的 GPU 数。例如，要使用一个支持 GPU 的容器映像，并且为每个 VM 分配 4 个 Nvidia Telsa K80 GPU，您可以指定：

    (aip.AcceleratorType.NVIDIA_TESLA_K80, 4)

否则指定 `(None, None)`，以在 CPU 上运行容器映像。

了解更多关于[您地区的硬件加速器支持](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)。

*注*: TF 在 2.3 之前的版本对 GPU 支持将无法加载本教程中的自定义模型。这是一个已知问题，已在 TF 2.3 中修复。这是由在服务函数中生成的静态图操作引起的。如果您在自己的自定义模型上遇到此问题，请使用支持 GPU 的 TF 2.3 容器映像。

In [ ]:
import os

if os.getenv("IS_TESTING_TRAIN_GPU"):
    TRAIN_GPU, TRAIN_NGPU = (
        aip.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_TRAIN_GPU")),
    )
else:
    TRAIN_GPU, TRAIN_NGPU = (None, None)

if os.getenv("IS_TESTING_DEPLOY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aip.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPLOY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (None, None)

设置预构建的容器

为训练和预测设置预构建的Docker容器镜像。

有关最新列表，请参阅[用于训练的预构建容器](https://cloud.google.com/ai-platform-unified/docs/training/pre-built-containers)。

有关最新列表，请参阅[用于预测的预构建容器](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。

In [ ]:
TRAIN_VERSION = "scikit-learn-cpu.0-23"
DEPLOY_VERSION = "sklearn-cpu.0-23"

TRAIN_IMAGE = "{}-docker.pkg.dev/vertex-ai/training/{}:latest".format(
    REGION.split("-")[0], TRAIN_VERSION
)
DEPLOY_IMAGE = "{}-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(
    REGION.split("-")[0], DEPLOY_VERSION
)

设置机器类型

接下来，设置用于训练的机器类型。

- 将变量`TRAIN_COMPUTE`设置为配置用于训练的虚拟机的计算资源。
 - `机器类型`
     - `n1-standard`: 每个vCPU 3.75GB内存。
     - `n1-highmem`: 每个vCPU 6.5GB内存。
     - `n1-highcpu`: 每个vCPU 0.9GB内存。
 - `vCPUs`: 数量为\[2, 4, 8, 16, 32, 64, 96\]

*注意: 以下类型不支持训练:*

 - `standard`: 2个vCPUs
 - `highcpu`: 2, 4和8个vCPUs

*注意: 您也可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU*。

In [ ]:
if os.getenv("IS_TESTING_TRAIN_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_TRAIN_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

## scikit-learn 训练简介

一旦你已经训练好一个 scikit-learn 模型，你会想要将它保存到一个云存储位置，这样后续就可以上传到 `Vertex AI Model` 资源中。Scikit-learn 软件包不支持将模型保存到云存储位置。相反，你需要按照以下步骤将模型保存到云存储位置。

1. 将内存中的模型以 pickle 格式（如 model.pkl）保存到本地文件系统中。
2. 创建一个云存储存储客户端。
3. 使用云存储存储客户端，将 pickle 文件作为一个 blob 上传到指定的云存储位置。

*注意*：你可以使用 Scikit-learn 模型进行超参数调整。

### 检查培训包

#### 包布局

在开始培训之前，您将查看如何为自定义培训任务组装Python包。解压缩后，包含以下目录/文件布局。

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

文件`setup.cfg`和`setup.py`是将包安装到Docker镜像的操作环境中的说明。

文件`trainer/task.py`是执行自定义培训任务的Python脚本。*注意*，当我们在工作池规范中引用它时，我们将目录斜杠替换为点(`trainer.task`)并删除文件后缀(`.py`)。

#### 包组装

在接下来的单元格中，您将组装培训包。

In [ ]:
# Make folder for Python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\ntag_build =\n\ntag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\nsetuptools.setup(\n\n    install_requires=[\n\n        'wget',\n\n        'cloudml-hypertune',\n\n    ],\n\n    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\nName: News Aggregation text classification\n\nVersion: 0.0.0\n\nSummary: Demostration training script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: aferlitsch@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

### 为Python培训套件创建任务脚本

接下来，您需要为驱动培训套件的`task.py`脚本。一些值得注意的步骤包括：

- 命令行参数:
    - `model-dir`：保存经训练的模型的位置。在使用Vertex AI自定义训练时，该位置将在环境变量`AIP_MODEL_DIR`中指定，
    - `dataset_url`：要下载的数据集的位置。
    - `alpha`：超参数
- 数据预处理（`get_data()`）：
    - 下载数据集并拆分为训练集和测试集。
- 模型架构（`get_model()`）：
    - 构建相应的模型架构。
- 训练（`train_model()`）：
    - 训练模型
- 评估（`evaluate_model()`）：
    - 评估模型。
    - 如果进行超参数调整，则报告准确度指标。
- 模型工件保存
    - 将模型工件和评估指标保存在由`model-dir`指定的Cloud Storage位置。
    - *注意*：GCSFuse（`/gcs`）用于在Cloud Storage存储桶上执行文件系统操作。

In [ ]:
%%writefile custom/trainer/task.py
import argparse
import logging
import os
import pickle
import zipfile
from typing import List, Tuple

import pandas as pd
import wget
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
import hypertune

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv('AIP_MODEL_DIR'), type=str, help='Model dir.')
parser.add_argument("--dataset-url", dest="dataset_url",
                    type=str, help="Download url for the training data.")
parser.add_argument('--alpha', dest='alpha',
                    default=1.0, type=float,
                    help='Alpha parameters for MultinomialNB')
args = parser.parse_args()

logging.getLogger().setLevel(logging.INFO)

def get_data(url: str, test_size: float = 0.2) -> Tuple[List, List, List, List]:
    logging.info("Downloading training data from: {}".format(args.dataset_url))

    zip_filepath = wget.download(url, out=".")

    with zipfile.ZipFile(zip_filepath, "r") as zf:
        zf.extract(path=".", member="newsCorpora.csv")

    COLUMN_NAMES = ["id", "title", "url", "publisher",
                    "category", "story", "hostname", "timestamp"]

    dataframe = pd.read_csv(
        "newsCorpora.csv", delimiter="	", names=COLUMN_NAMES, index_col=0
    )

    train, test = train_test_split(dataframe, test_size=test_size)

    x_train, y_train = train["title"].values, train["category"].values
    x_test, y_test = test["title"].values, test["category"].values

    return x_train, y_train, x_test, y_test

def get_model():
    logging.info("Build model ...")
    model = Pipeline([
                ("vectorizer", CountVectorizer()),
                ("tfidf", TfidfTransformer()),
                ("naivebayes", MultinomialNB(alpha=args.alpha)),
        ])
    return model

def train_model(model: Pipeline, X_train: List, y_train: List, X_test: List, y_test: List
) -> Pipeline:
    logging.info("Training started ...")
    model.fit(X_train, y_train)
    logging.info("Training completed")
    return model

def evaluate_model(model: Pipeline, X_train: List, y_train: List, X_test: List, y_test: List
) -> float:
    score = model.score(X_test, y_test)
    logging.info(f"Evaluation completed with model score: {score}")

    # report metric for hyperparameter tuning
    hpt = hypertune.HyperTune()
    hpt.report_hyperparameter_tuning_metric(
        hyperparameter_metric_tag='accuracy',
        metric_value=score
    )
    return score


def export_model_to_gcs(fitted_pipeline: Pipeline, gcs_uri: str) -> str:
    """Exports trained pipeline to GCS
    Parameters:
            fitted_pipeline (sklearn.pipelines.Pipeline): the Pipeline object
                with data already fitted (trained pipeline object).
            gcs_uri (str): GCS path to store the trained pipeline
                i.e gs://example_bucket/training-job.
    Returns:
            export_path (str): Model GCS location
    """
    # Upload model artifact to Cloud Storage
    artifact_filename = 'model.pkl'
    storage_path = os.path.join(gcs_uri, artifact_filename)

    # Save model artifact to local filesystem (doesn't persist)
    with open(storage_path, 'wb') as model_file:
        pickle.dump(fitted_pipeline, model_file)


def export_evaluation_report_to_gcs(report: str, gcs_uri: str) -> None:
    """
    Exports training job report to GCS
    Parameters:
        report (str): Full report in text to sent to GCS
        gcs_uri (str): GCS path to store the report
            i.e gs://example_bucket/training-job
    """

    # Upload model artifact to Cloud Storage
    artifact_filename = 'report.txt'
    storage_path = os.path.join(gcs_uri, artifact_filename)

    # Save model artifact to local filesystem (doesn't persist)
    with open(storage_path, 'w') as report_file:
        report_file.write(report)


logging.info("Starting custom training job.")

data = get_data(args.dataset_url)
model = get_model()
model = train_model(model, *data)
score = evaluate_model(model, *data)

# export model to gcs using GCSFuse
logging.info("Exporting model artifacts ...")
gs_prefix = 'gs://'
gcsfuse_prefix = '/gcs/'
if args.model_dir.startswith(gs_prefix):
    args.model_dir = args.model_dir.replace(gs_prefix, gcsfuse_prefix)
    dirpath = os.path.split(args.model_dir)[0]
    if not os.path.isdir(dirpath):
        os.makedirs(dirpath)

export_model_to_gcs(model, args.model_dir)
export_evaluation_report_to_gcs(str(score), args.model_dir)
logging.info(f"Exported model artifacts to GCS bucket: {args.model_dir}")

请将培训脚本存储在您的云存储桶中

接下来，您将培训文件夹打包成压缩的tar文件，并将其存储在您的云存储桶中。

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_URI/trainer_newsaggr.tar.gz

### 创建并运行自定义训练作业

为了训练一个自定义模型，您需要执行两个步骤：1）创建一个自定义训练作业，以及2）运行这个作业。

#### 创建自定义训练作业

使用`CustomTrainingJob`类创建一个自定义训练作业，需要提供以下参数：

- `display_name`：自定义训练作业的可读名称。
- `container_uri`：训练容器镜像。

- `python_package_gcs_uri`：Python训练包的位置，作为一个tarball。
- `python_module_name`：Python包中训练脚本的相对路径。
- `model_serving_container_uri`：部署模型的容器镜像。

*注意：* 没有要求参数。您可以在Python包的`setup.py`脚本中指定任何依赖。

In [ ]:
DISPLAY_NAME = "newsaggr_" + TIMESTAMP

job = aip.CustomPythonPackageTrainingJob(
    display_name=DISPLAY_NAME,
    python_package_gcs_uri=f"{BUCKET_URI}/trainer_newsaggr.tar.gz",
    python_module_name="trainer.task",
    container_uri=TRAIN_IMAGE,
    model_serving_container_image_uri=DEPLOY_IMAGE,
    project=PROJECT_ID,
)

### 准备您的命令行参数

现在为您的自定义训练容器定义命令行参数：

- `args`：要传递给作为容器入口点设置的可执行文件的命令行参数。
  - `--model-dir`：对于我们的演示，我们使用此命令行参数来指定存储模型工件的位置。
      - 直接传递：您将Cloud Storage位置作为命令行参数传递给训练脚本（设置变量 `DIRECT = True`），或
      - 间接传递：服务将Cloud Storage位置作为环境变量 `AIP_MODEL_DIR` 传递给您的训练脚本（设置变量 `DIRECT = False`）。在这种情况下，您在作业规范中告诉服务模型工件的位置。
  - `--dataset-url`：要下载的数据集的位置。
  - `--alpha`：可调整的超参数

In [ ]:
MODEL_DIR = "{}/{}".format(BUCKET_URI, TIMESTAMP)
DATASET_URL = "https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip"

DIRECT = False

if DIRECT:
    CMDARGS = [
        "--alpha=" + str(0.9),
        "--dataset-url=" + DATASET_URL,
        "--model_dir=" + MODEL_DIR,
    ]
else:
    CMDARGS = ["--alpha=" + str(0.9), "--dataset-url=" + DATASET_URL]

#### 运行自定义训练作业

接下来，通过调用方法`run`，使用以下参数运行自定义作业来启动训练作业：

- `model_display_name`：`Model`资源的可读性名称。
- `args`：要传递给训练脚本的命令行参数。
- `replica_count`：用于训练的计算实例的数量（replica_count = 1 表示单节点训练）。
- `machine_type`：计算实例的机器类型。
- `accelerator_type`：硬件加速器类型。
- `accelerator_count`：要附加到工作人员副本的加速器数量。
- `base_output_dir`：写入模型工件的 Cloud 存储位置。
- `sync`：是否阻塞直至作业完成。

In [ ]:
if TRAIN_GPU:
    model = job.run(
        model_display_name="newsaggr_" + TIMESTAMP,
        args=CMDARGS,
        replica_count=1,
        machine_type=TRAIN_COMPUTE,
        accelerator_type=TRAIN_GPU.name,
        accelerator_count=TRAIN_NGPU,
        base_output_dir=MODEL_DIR,
        sync=False,
    )
else:
    model = job.run(
        model_display_name="newsaggr_" + TIMESTAMP,
        args=CMDARGS,
        replica_count=1,
        machine_type=TRAIN_COMPUTE,
        base_output_dir=MODEL_DIR,
        sync=False,
    )

model_path_to_deploy = MODEL_DIR

列出一个定制的培训工作

In [ ]:
_job = job.list(filter=f"display_name={DISPLAY_NAME}")
print(_job)

### 等待自定义训练作业完成

接下来，等待自定义训练作业完成。另外，可以在`run()`方法中将参数`sync`设置为`True`，以阻塞直到自定义训练作业完成。

In [ ]:
model.wait()

### 删除一个自定义训练任务

在训练任务完成后，您可以使用方法 `delete()` 删除训练任务。在完成之前，训练任务可以使用方法 `cancel()` 取消。

In [ ]:
job.delete()

清理工作

要清理此项目中使用的所有Google Cloud资源，您可以删除用于教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在此教程中创建的个别资源：

- 模型
- 云存储桶

In [ ]:
# Delete the model using the Vertex model object
model.delete()

delete_bucket = False
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI